# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: _LORA_
* Model: _distilbert-base-uncased_
* Evaluation approach: _Huggingface Trainer: evaluate_
* Fine-tuning dataset: _sms_spam_

## 1. Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
! pip install evaluate

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.3 MB/s eta 0:00:0000:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
! pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 60.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 38.8 MB/s eta 0:00:00


In [1]:
! pip install -q "datasets==2.15.0"

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Imports

In [2]:
import pandas as pd
import numpy as np
import torch

## Dataset
### Loading

In [3]:
# Load the sms_spam dataset
# See: https://huggingface.co/datasets/sms_spam

from datasets import load_dataset

# The sms_spam dataset only has a train split, so we use the train_test_split method to split it into train and test
dataset = load_dataset("sms_spam", split="train").train_test_split(
    test_size=0.2, shuffle=True, seed=23
)

splits = ["train", "test"]

# View the dataset characteristics
dataset["train"]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5574 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label'],
    num_rows: 4459
})

### EDA and Pre-Process

In [4]:
# Checking random example to see if it's a spam or not?
dataset["train"][77]

{'sms': 'SMS AUCTION You have won a Nokia 7250i. This is what you get when you win our FREE auction. To take part send Nokia to 86021 now. HG/Suite342/2Lands Row/W1JHL 16+\n',
 'label': 1}

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Let's use a lambda function to tokenize all the examples
tokenized_dataset = {}
for split in splits:
    tokenized_dataset[split] = dataset[split].map(
        lambda x: tokenizer(x["sms"], truncation=True), batched=True
    )

# Checking the available columns in the dataset
tokenized_dataset["train"]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/4459 [00:00<?, ? examples/s]

Map:   0%|          | 0/1115 [00:00<?, ? examples/s]

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 4459
})

### Loading a pretrained HuggingFace foundation model

In [6]:
# In this case we are doing a full fine tuning, so we will want to unfreeze all parameters.

from transformers import AutoModelForSequenceClassification

full_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2,
    id2label={0: "not spam", 1: "spam"},
    label2id={"not spam": 0, "spam": 1},
)

# Unfreeze all the model parameters.
# Hint: Check the documentation at https://huggingface.co/transformers/v4.2.2/training.html
for param in full_model.parameters():
    param.requires_grad = True

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
print(full_model)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

### Training or Fine Tuning (if necessary)

In [8]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=full_model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        
        num_train_epochs=2,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.evaluate()
# trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6589354872703552,
 'eval_accuracy': 0.8538116591928251,
 'eval_runtime': 2.8086,
 'eval_samples_per_second': 397.002,
 'eval_steps_per_second': 24.924}

In [9]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.048534,0.989238
2,0.054000,0.051910,0.988341


Checkpoint destination directory ./data/spam_not_spam/checkpoint-279 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/spam_not_spam/checkpoint-558 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=558, training_loss=0.05068818343582974, metrics={'train_runtime': 68.1073, 'train_samples_per_second': 130.94, 'train_steps_per_second': 8.193, 'total_flos': 144666559425588.0, 'train_loss': 0.05068818343582974, 'epoch': 2.0})

### Evaluate the full fine-tuned model

In [10]:
trainer.evaluate()

{'eval_loss': 0.04853416979312897,
 'eval_accuracy': 0.989237668161435,
 'eval_runtime': 2.2697,
 'eval_samples_per_second': 491.259,
 'eval_steps_per_second': 30.841,
 'epoch': 2.0}

In [11]:
# Making a dataframe with the predictions and the text and the labels
items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 77, 131, 243, 303, 450, 587]
)

results = trainer.predict(items_for_manual_review)
df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,sms,predictions,labels
0,Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n,0,0
1,Happy new years melody!\n,0,0
2,My superior telling that friday is leave for all other department except ours:)so it will be leave for you:)any way call waheed fathima hr and conform it:)\n,0,0
3,"I just got home babe, are you still awake ?\n",0,0
4,"URGENT! Your mobile No 077xxx WON a £2,000 Bonus Caller Prize on 02/06/03! This is the 2nd attempt to reach YOU! Call 09066362206 ASAP! BOX97N7QP, 150ppm\n",1,1
5,", ow u dey.i paid 60,400thousad.i told u would call . \n",0,0
6,Anything lor is she coming?\n,0,0
7,18 days to Euro2004 kickoff! U will be kept informed of all the latest news and results daily. Unsubscribe send GET EURO STOP to 83222.\n,1,1


## 2. Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

### Bulding a PEFT model

In [38]:
from peft import get_peft_config, get_peft_model, LoraConfig, TaskType

peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, 
    inference_mode=False, 
    r=8, 
    target_modules=["q_lin", "v_lin"],
    lora_alpha=32, 
    lora_dropout=0.1
)

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased"
)

# creating a PEFT model
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,331,716 || all params: 67,694,596 || trainable%: 1.967241225577297


### Training/fine-tuning the PEFT model

In [39]:
from transformers import DataCollatorWithPadding, Trainer, TrainingArguments

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


# The HuggingFace Trainer class handles the training and eval loop for PyTorch for us.
# Read more about it here https://huggingface.co/docs/transformers/main_classes/trainer
trainer = Trainer(
    model=peft_model,
    args=TrainingArguments(
        output_dir="./data/spam_not_spam",
        # Set the learning rate
        learning_rate=2e-5,
        # Set the per device train batch size and eval batch size
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        # Evaluate and save the model after each epoch
        evaluation_strategy="epoch",
        save_strategy="epoch",
        
        num_train_epochs=1,
        weight_decay=0.01,
        load_best_model_at_end=True,
    ),
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)
trainer.evaluate()
# trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6834543943405151,
 'eval_accuracy': 0.6762331838565022,
 'eval_runtime': 2.4598,
 'eval_samples_per_second': 453.287,
 'eval_steps_per_second': 28.458}

In [40]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.161792,0.938117


Checkpoint destination directory ./data/spam_not_spam/checkpoint-279 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=279, training_loss=0.29378167702732977, metrics={'train_runtime': 20.7387, 'train_samples_per_second': 215.009, 'train_steps_per_second': 13.453, 'total_flos': 73790878281600.0, 'train_loss': 0.29378167702732977, 'epoch': 1.0})

In [42]:
trainer.evaluate()

{'eval_loss': 0.16179241240024567,
 'eval_accuracy': 0.9381165919282511,
 'eval_runtime': 2.445,
 'eval_samples_per_second': 456.037,
 'eval_steps_per_second': 28.63,
 'epoch': 1.0}

In [17]:
# Make a dataframe with the predictions and the text and the labels
items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 77, 131, 243, 303, 450, 587]
)

results = trainer.predict(items_for_manual_review)
df = pd.DataFrame(
    {
        "sms": [item["sms"] for item in items_for_manual_review],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,sms,predictions,labels
0,Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n,0,0
1,Happy new years melody!\n,0,0
2,My superior telling that friday is leave for all other department except ours:)so it will be leave for you:)any way call waheed fathima hr and conform it:)\n,0,0
3,"I just got home babe, are you still awake ?\n",0,0
4,"URGENT! Your mobile No 077xxx WON a £2,000 Bonus Caller Prize on 02/06/03! This is the 2nd attempt to reach YOU! Call 09066362206 ASAP! BOX97N7QP, 150ppm\n",1,1
5,", ow u dey.i paid 60,400thousad.i told u would call . \n",0,0
6,Anything lor is she coming?\n,0,0
7,18 days to Euro2004 kickoff! U will be kept informed of all the latest news and results daily. Unsubscribe send GET EURO STOP to 83222.\n,1,1


### Saving the PEFT model

In [18]:
#Saving the trained model
peft_model.save_pretrained("./peft_model")

## 3. Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

### Load the saved PEFT model

In [19]:
from peft import PeftModel, PeftConfig, AutoPeftModelForSequenceClassification
from transformers import AutoModelForSequenceClassification

In [20]:
peft_model_id = "./peft_model"
config = PeftConfig.from_pretrained(peft_model_id)

In [21]:
model = AutoPeftModelForSequenceClassification.from_pretrained(peft_model_id)
model

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(
                  in_features=768, out_features=768, bias=True
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_features=768, out_features=8, bias=Fals

In [26]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

### Testing

In [27]:
#https://huggingface.co/docs/transformers/main/en/model_doc/distilbert#transformers.DistilBertForSequenceClassification
x = tokenized_dataset["test"].select(
    [0, 1, 77, 131, 243, 303, 450, 587]
)

print(x)

for i in x:
    print(i['input_ids'])
    print(i)
    break

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 8
})
[101, 9805, 2361, 1012, 1012, 1012, 4931, 2059, 2028, 2154, 2006, 10424, 2072, 2057, 2064, 3198, 2771, 4213, 1998, 25871, 25811, 2202, 2681, 2175, 13173, 11045, 102]
{'sms': 'Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n', 'label': 0, 'input_ids': [101, 9805, 2361, 1012, 1012, 1012, 4931, 2059, 2028, 2154, 2006, 10424, 2072, 2057, 2064, 3198, 2771, 4213, 1998, 25871, 25811, 2202, 2681, 2175, 13173, 11045, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [28]:
# Make a dataframe with the predictions and the text and the labels
items_for_manual_review = tokenized_dataset["test"].select(
    [0, 1, 77, 131, 243, 303, 450, 587]
)

predictions = []
for i in items_for_manual_review:
    input_tokens = tokenizer(i['sms'], return_tensors="pt")
    #print(input_tokens)
    with torch.no_grad():
        logits = model(**input_tokens).logits
        predicted_class_id = logits.argmax().item()
        predictions.append(predicted_class_id)

In [29]:
print(predictions)

[0, 0, 0, 0, 0, 0, 0, 0]


In [30]:
df_preds = pd.DataFrame(
    {
        "sms": [item["sms"] for item in items_for_manual_review],
        #"predictions": results.predictions.argmax(axis=1),
        "predictions": predictions,
        #"labels": results.label_ids,
        "labels": [item["label"] for item in items_for_manual_review],
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df_preds

,sms,predictions,labels
0,Yup... Hey then one day on fri we can ask miwa and jiayin take leave go karaoke \n,0,0
1,Happy new years melody!\n,0,0
2,My superior telling that friday is leave for all other department except ours:)so it will be leave for you:)any way call waheed fathima hr and conform it:)\n,0,0
3,"I just got home babe, are you still awake ?\n",0,0
4,"URGENT! Your mobile No 077xxx WON a £2,000 Bonus Caller Prize on 02/06/03! This is the 2nd attempt to reach YOU! Call 09066362206 ASAP! BOX97N7QP, 150ppm\n",0,1
5,", ow u dey.i paid 60,400thousad.i told u would call . \n",0,0
6,Anything lor is she coming?\n,0,0
7,18 days to Euro2004 kickoff! U will be kept informed of all the latest news and results daily. Unsubscribe send GET EURO STOP to 83222.\n,0,1


### Evaluate the fine-tuned model

In [31]:
labels = [item["label"] for item in items_for_manual_review]
labels

[0, 0, 0, 0, 1, 0, 0, 1]

In [32]:
import evaluate
accuracy_metric = evaluate.load("accuracy")
accuracy_metric

EvaluationModule(name: "accuracy", module_type: "metric", features: {'predictions': Value(dtype='int32', id=None), 'references': Value(dtype='int32', id=None)}, usage: """
Args:
    predictions (`list` of `int`): Predicted labels.
    references (`list` of `int`): Ground truth labels.
    normalize (`boolean`): If set to False, returns the number of correctly classified samples. Otherwise, returns the fraction of correctly classified samples. Defaults to True.
    sample_weight (`list` of `float`): Sample weights Defaults to None.

Returns:
    accuracy (`float` or `int`): Accuracy score. Minimum possible value is 0. Maximum possible value is 1.0, or the number of examples input, if `normalize` is set to `True`.. A higher score means higher accuracy.

Examples:

    Example 1-A simple example
        >>> accuracy_metric = evaluate.load("accuracy")
        >>> results = accuracy_metric.compute(references=[0, 1, 2, 0, 1, 2], predictions=[0, 1, 1, 2, 1, 0])
        >>> print(results)
    

In [33]:
results = accuracy_metric.compute(references=labels, predictions=predictions)
print(results)

{'accuracy': 0.75}
